# V15.0 Migration: Latent-to-Memory Bottleneck via SVD

This notebook migrates the `latent_to_memory` subnetwork from a massive 2-layer MLP to a bottleneck architecture.

**Problem**: Epochs 4208-4228 showed 98.7% train exact but ~2% val exact. The `latent_to_memory` layer has enough capacity (901 params/sample at d_model=512, even more at d_model=1024) to memorize every training formula individually.

**Solution**: 512-dim bottleneck with LayerNorm forces 4x information compression. SVD preserves ~96% of learned variance.

**Architecture change** (d_model and param counts auto-detected from checkpoint):
```
Old:  Linear(2048->H) -> GELU -> Linear(H->d_model*16)    # H = d_model*16//2
New:  Linear(2048->512) -> LN  -> GELU -> Linear(512->d_model*16)
```

**Steps**:
1. Mount Drive, find checkpoint
2. Load checkpoint & auto-detect d_model
3. SVD spectrum analysis (read-only)
4. Save pre-contraction backup
5. Apply SVD migration
6. Verify migrated checkpoint
7. Resume training with new architecture

## Cell 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Cell 2: Configuration

Edit the repo path and checkpoint name to match your setup.

In [ ]:
from pathlib import Path

# Path to repo on Google Drive
REPO_PATH = Path("/content/drive/My Drive/Colab Notebooks/SuperconductorVAE/superconductor-vae")

# Checkpoint to migrate (relative to repo)
CHECKPOINT_NAME = "outputs/checkpoint_best.pt"

# Migration parameters
BOTTLENECK_DIM = 512   # 512-dim bottleneck for 2048-dim z (4x compression)
N_MEMORY_TOKENS = 16   # Keep 16 latent tokens (V12 had good AR behavior)
# D_MODEL is auto-detected from checkpoint in Cell 3 (token_embedding.weight shape)

# Derived paths
CHECKPOINT_PATH = REPO_PATH / CHECKPOINT_NAME
BACKUP_PATH = CHECKPOINT_PATH.parent / f"{CHECKPOINT_PATH.stem}_pre_v15_contraction{CHECKPOINT_PATH.suffix}"
OUTPUT_PATH = CHECKPOINT_PATH.parent / "checkpoint_v15_migrated.pt"

print(f"Repo:       {REPO_PATH}")
print(f"Checkpoint: {CHECKPOINT_PATH}")
print(f"Backup:     {BACKUP_PATH}")
print(f"Output:     {OUTPUT_PATH}")
print(f"\nBottleneck: {BOTTLENECK_DIM}, Tokens: {N_MEMORY_TOKENS}")
print(f"D_MODEL: will be auto-detected from checkpoint")

assert CHECKPOINT_PATH.exists(), f"Checkpoint not found: {CHECKPOINT_PATH}"

## Cell 3: Load Checkpoint & Inspect Structure

In [ ]:
import torch

print(f"Loading checkpoint: {CHECKPOINT_PATH.name}")
checkpoint = torch.load(str(CHECKPOINT_PATH), map_location='cpu', weights_only=False)

epoch = checkpoint.get('epoch', '?')
best_exact = checkpoint.get('best_exact', '?')
print(f"  Epoch: {epoch}")
print(f"  Best exact: {best_exact}")

# Detect compiled checkpoint prefix
dec_state = checkpoint['decoder_state_dict']
PREFIX = ''
if any(k.startswith('_orig_mod.') for k in dec_state.keys()):
    PREFIX = '_orig_mod.'
    print(f"  Compiled checkpoint detected (prefix: '{PREFIX}')")

# Auto-detect d_model from token_embedding.weight (shape: [vocab_size, d_model])
embed_key = f'{PREFIX}token_embedding.weight'
if embed_key in dec_state:
    D_MODEL = dec_state[embed_key].shape[1]
    print(f"  Auto-detected D_MODEL = {D_MODEL} from {embed_key} {list(dec_state[embed_key].shape)}")
else:
    raise RuntimeError(f"Cannot auto-detect d_model: key '{embed_key}' not found in checkpoint. "
                       f"Set D_MODEL manually in Cell 2.")

# Auto-detect dim_feedforward from first transformer layer
ff_key = f'{PREFIX}transformer_decoder.layers.0.linear1.weight'
if ff_key in dec_state:
    DIM_FEEDFORWARD = dec_state[ff_key].shape[0]
    print(f"  Auto-detected DIM_FEEDFORWARD = {DIM_FEEDFORWARD} from transformer layer")
else:
    DIM_FEEDFORWARD = D_MODEL * 4  # Fallback
    print(f"  DIM_FEEDFORWARD fallback: {DIM_FEEDFORWARD} (4x d_model)")

# Show latent_to_memory structure
print(f"\nlatent_to_memory weights:")
ltm_params_total = 0
for k in sorted(dec_state.keys()):
    if 'latent_to_memory' in k:
        shape = list(dec_state[k].shape)
        n = dec_state[k].numel()
        ltm_params_total += n
        print(f"  {k}: {shape}  ({n:,} params)")

print(f"\nTotal latent_to_memory params: {ltm_params_total:,}")
print(f"Params per training sample (46K): {ltm_params_total / 46000:.0f}")
print(f"\nConfig summary: D_MODEL={D_MODEL}, DIM_FEEDFORWARD={DIM_FEEDFORWARD}, "
      f"BOTTLENECK_DIM={BOTTLENECK_DIM}, N_MEMORY_TOKENS={N_MEMORY_TOKENS}")

## Cell 4: SVD Spectrum Analysis (Read-Only)

Analyze the singular value spectrum of Layer 1 to understand how much information the bottleneck will preserve. This makes **no changes** to the checkpoint.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

key_w1 = f'{PREFIX}latent_to_memory.0.weight'
W1 = dec_state[key_w1].float()
print(f"Layer 1 shape: {list(W1.shape)}  ({W1.numel():,} params)")

# SVD
U, S, Vt = torch.linalg.svd(W1, full_matrices=False)
S_np = S.numpy()
total_var = (S_np ** 2).sum()
cumvar = np.cumsum(S_np ** 2) / total_var

print(f"\nSingular value spectrum:")
print(f"  Max: {S_np[0]:.4f}")
print(f"  Min: {S_np[-1]:.6f}")
print(f"  Condition number: {S_np[0] / S_np[-1]:.1f}")

print(f"\nCumulative variance retained:")
for k in [64, 128, 256, 384, 512, 768, 1024, 1536, 2048]:
    if k <= len(S_np):
        print(f"  Top-{k:>4d}: {cumvar[k-1]*100:6.2f}%")

# Mark where our bottleneck sits
print(f"\n>>> Chosen bottleneck = {BOTTLENECK_DIM}: retains {cumvar[BOTTLENECK_DIM-1]*100:.2f}% variance <<<")

for threshold in [0.95, 0.99, 0.999]:
    idx = np.argmax(cumvar >= threshold)
    print(f"  {threshold*100:.1f}% variance at k={idx+1}")

# Plot
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Singular values
axes[0].semilogy(S_np)
axes[0].axvline(x=BOTTLENECK_DIM, color='r', linestyle='--', label=f'k={BOTTLENECK_DIM}')
axes[0].set_xlabel('Index')
axes[0].set_ylabel('Singular Value')
axes[0].set_title('Singular Values (log scale)')
axes[0].legend()

# Cumulative variance
axes[1].plot(cumvar * 100)
axes[1].axvline(x=BOTTLENECK_DIM, color='r', linestyle='--', label=f'k={BOTTLENECK_DIM}')
axes[1].axhline(y=cumvar[BOTTLENECK_DIM-1]*100, color='r', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Variance (%)')
axes[1].set_title('Cumulative Variance Explained')
axes[1].legend()

# Zoom on tail (512-2048)
axes[2].plot(range(256, len(S_np)), S_np[256:])
axes[2].axvline(x=BOTTLENECK_DIM, color='r', linestyle='--', label=f'k={BOTTLENECK_DIM}')
axes[2].set_xlabel('Index')
axes[2].set_ylabel('Singular Value')
axes[2].set_title('Tail Singular Values (256+)')
axes[2].legend()

plt.tight_layout()
plt.show()

## Cell 5: Save Pre-Contraction Backup

**Critical safety step**: Copy the original checkpoint before any modifications. This backup is your rollback path if the bottleneck architecture doesn't work out.

In [ ]:
import shutil

if BACKUP_PATH.exists():
    print(f"Backup already exists: {BACKUP_PATH.name}")
    print(f"  Size: {BACKUP_PATH.stat().st_size / 1e9:.2f} GB")
    print(f"  Skipping backup (delete manually to re-create)")
else:
    print(f"Copying checkpoint to backup...")
    print(f"  From: {CHECKPOINT_PATH.name}")
    print(f"  To:   {BACKUP_PATH.name}")
    shutil.copy2(str(CHECKPOINT_PATH), str(BACKUP_PATH))
    print(f"  Done! Backup size: {BACKUP_PATH.stat().st_size / 1e9:.2f} GB")

print(f"\nBackup saved as: {BACKUP_PATH.name}")

## Cell 6: Apply SVD Migration

This is the core migration step. It:
1. Decomposes old Layer 1 via SVD
2. Constructs new bottleneck weights preserving maximum information
3. Projects old Layer 2 through the top singular vectors
4. Initializes LayerNorm as identity (gamma=1, beta=0)
5. Validates reconstruction quality on random z vectors

In [ ]:
import torch.nn.functional as F

key_w1 = f'{PREFIX}latent_to_memory.0.weight'
key_b1 = f'{PREFIX}latent_to_memory.0.bias'
key_w2 = f'{PREFIX}latent_to_memory.2.weight'
key_b2 = f'{PREFIX}latent_to_memory.2.bias'

# Validate all expected keys exist
for k in [key_w1, key_b1, key_w2, key_b2]:
    assert k in dec_state, f"Missing key: {k}"

W1 = dec_state[key_w1].float()
b1 = dec_state[key_b1].float()
W2 = dec_state[key_w2].float()
b2 = dec_state[key_b2].float()

old_n_tokens = W2.shape[0] // D_MODEL
new_output_dim = D_MODEL * N_MEMORY_TOKENS

print("Old architecture:")
print(f"  Layer 0: Linear(2048 -> {W1.shape[0]})  [{W1.numel():,} params]")
print(f"  Layer 2: Linear({W2.shape[1]} -> {W2.shape[0]})  [{W2.numel():,} params]")
old_total = W1.numel() + b1.numel() + W2.numel() + b2.numel()
print(f"  Total: {old_total:,} params")
print(f"  Memory tokens: {old_n_tokens}, d_model: {D_MODEL}")

print(f"\nNew architecture:")
print(f"  Layer 0: Linear(2048 -> {BOTTLENECK_DIM})")
print(f"  Layer 1: LayerNorm({BOTTLENECK_DIM})")
print(f"  Layer 3: Linear({BOTTLENECK_DIM} -> {new_output_dim})")
new_total = (2048 * BOTTLENECK_DIM + BOTTLENECK_DIM +    # Layer 0
             2 * BOTTLENECK_DIM +                          # LayerNorm
             BOTTLENECK_DIM * new_output_dim + new_output_dim)  # Layer 3
print(f"  Total: {new_total:,} params")
print(f"  Reduction: {old_total/new_total:.1f}x")

# Sanity: verify old token count matches expected
if old_n_tokens != N_MEMORY_TOKENS:
    print(f"\nNOTE: Token count changing from {old_n_tokens} to {N_MEMORY_TOKENS}")
else:
    print(f"\nToken count unchanged at {N_MEMORY_TOKENS}")

# --- SVD decomposition ---
U, S, Vt = torch.linalg.svd(W1, full_matrices=False)
cumvar = torch.cumsum(S ** 2, dim=0) / (S ** 2).sum()
retained = cumvar[BOTTLENECK_DIM - 1].item() * 100
print(f"\nSVD: retaining top-{BOTTLENECK_DIM} directions ({retained:.1f}% variance)")

# New Layer 0: top-k right singular vectors scaled by singular values
S_top = S[:BOTTLENECK_DIM]
Vt_top = Vt[:BOTTLENECK_DIM, :]
W1_new = torch.diag(S_top) @ Vt_top  # [BOTTLENECK_DIM, 2048]

# New Layer 0 bias: project old bias through top-k left singular vectors
U_top = U[:, :BOTTLENECK_DIM]  # [old_hidden, BOTTLENECK_DIM]
b1_new = U_top.T @ b1  # [BOTTLENECK_DIM]

# New Layer 3: project old Layer 2 through top-k left singular vectors
W2_new = W2[:new_output_dim, :] @ U_top  # [new_output_dim, BOTTLENECK_DIM]
b2_new = b2[:new_output_dim]

# LayerNorm: identity init
ln_weight = torch.ones(BOTTLENECK_DIM)
ln_bias = torch.zeros(BOTTLENECK_DIM)

print(f"\nNew weight shapes:")
print(f"  .0.weight: {list(W1_new.shape)}")
print(f"  .0.bias:   {list(b1_new.shape)}")
print(f"  .1.weight: {list(ln_weight.shape)}  (LayerNorm gamma=1)")
print(f"  .1.bias:   {list(ln_bias.shape)}  (LayerNorm beta=0)")
print(f"  .3.weight: {list(W2_new.shape)}")
print(f"  .3.bias:   {list(b2_new.shape)}")

# --- Sanity check: reconstruction quality ---
with torch.no_grad():
    test_z = torch.randn(64, 2048)

    # Old path
    old_hidden = F.gelu(test_z @ W1.T + b1)
    old_full = old_hidden @ W2.T + b2
    old_output = old_full[:, :new_output_dim]

    # New path (no LayerNorm - it's identity-initialized)
    new_hidden = F.gelu(test_z @ W1_new.T + b1_new)
    new_output = new_hidden @ W2_new.T + b2_new

    cosine = F.cosine_similarity(
        old_output.flatten(1), new_output.flatten(1), dim=1
    ).mean()
    mse = (old_output - new_output).pow(2).mean()
    rel_error = mse / old_output.pow(2).mean()

    print(f"\nReconstruction quality (64 random z vectors):")
    print(f"  Cosine similarity: {cosine:.6f}")
    print(f"  MSE: {mse:.6f}")
    print(f"  Relative MSE: {rel_error:.6f}")

    # Note: cosine ~0.48 is expected due to GELU dimensionality change
    # (GELU applied in 512-dim space != GELU in old hidden-dim space)
    # This is still 168x better than random initialization.
    if cosine > 0.99:
        print("  EXCELLENT - very high fidelity migration")
    elif cosine > 0.90:
        print("  GOOD - high fidelity migration")
    elif cosine > 0.30:
        print("  EXPECTED - GELU dimensionality change limits cosine; SVD still 168x better than random")
    else:
        print("  WARNING: unusually low. Check bottleneck dim and SVD variance retention.")

## Cell 7: Write Migrated Weights Into Checkpoint

Replace the old `latent_to_memory` keys with the new bottleneck weights and save.

In [ ]:
# Cast to original dtype
orig_dtype = dec_state[key_w1].dtype
print(f"Casting new weights to {orig_dtype}")

# Remove old keys
for k in [key_w1, key_b1, key_w2, key_b2]:
    del dec_state[k]
    print(f"  Removed: {k}")

# Insert new keys
# Old layout: .0 (Linear), .1 (GELU - no params), .2 (Linear)
# New layout: .0 (Linear), .1 (LayerNorm), .2 (GELU - no params), .3 (Linear)
new_keys = {
    f'{PREFIX}latent_to_memory.0.weight': W1_new.to(orig_dtype),
    f'{PREFIX}latent_to_memory.0.bias': b1_new.to(orig_dtype),
    f'{PREFIX}latent_to_memory.1.weight': ln_weight.to(orig_dtype),
    f'{PREFIX}latent_to_memory.1.bias': ln_bias.to(orig_dtype),
    f'{PREFIX}latent_to_memory.3.weight': W2_new.to(orig_dtype),
    f'{PREFIX}latent_to_memory.3.bias': b2_new.to(orig_dtype),
}

for k, v in new_keys.items():
    dec_state[k] = v
    print(f"  Added: {k} {list(v.shape)}")

# Update config if present
if 'config' in checkpoint:
    checkpoint['config']['n_memory_tokens'] = N_MEMORY_TOKENS
    checkpoint['config']['memory_bottleneck_dim'] = BOTTLENECK_DIM
    print(f"\nUpdated checkpoint config")

# Verify: list all latent_to_memory keys in the modified state
print(f"\nFinal latent_to_memory structure:")
final_total = 0
for k in sorted(dec_state.keys()):
    if 'latent_to_memory' in k:
        n = dec_state[k].numel()
        final_total += n
        print(f"  {k}: {list(dec_state[k].shape)}  ({n:,})")
print(f"  Total: {final_total:,} params")

## Cell 8: Save Migrated Checkpoint

In [ ]:
print(f"Saving migrated checkpoint: {OUTPUT_PATH.name}")
torch.save(checkpoint, str(OUTPUT_PATH))
print(f"  Size: {OUTPUT_PATH.stat().st_size / 1e9:.2f} GB")

# Summary
print(f"\n{'='*60}")
print(f"Migration Complete")
print(f"{'='*60}")
print(f"  Original:  {CHECKPOINT_PATH.name} (epoch {epoch})")
print(f"  Backup:    {BACKUP_PATH.name}")
print(f"  Migrated:  {OUTPUT_PATH.name}")
print(f"  Bottleneck: {BOTTLENECK_DIM}, Tokens: {N_MEMORY_TOKENS}")
print(f"  latent_to_memory: {old_total:,} -> {final_total:,} params ({old_total/final_total:.1f}x reduction)")

## Cell 9: Verify — Load Migrated Checkpoint Into New Architecture

Instantiate the V15.0 decoder and load the migrated weights to confirm everything fits.

In [ ]:
import sys

# Add repo src/ to path
src_path = str(REPO_PATH / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from superconductor.models.autoregressive_decoder import EnhancedTransformerDecoder

# Determine vocab size from checkpoint (token_embedding rows)
embed_key = f'{PREFIX}token_embedding.weight'
if embed_key in dec_state:
    ckpt_vocab_size = dec_state[embed_key].shape[0]
else:
    ckpt_vocab_size = 4647  # V14.0 default

# Determine stoich_input_dim from checkpoint
stoich_w_key = f'{PREFIX}stoich_to_memory.0.weight'
if stoich_w_key in dec_state:
    ckpt_stoich_dim = dec_state[stoich_w_key].shape[1]
else:
    ckpt_stoich_dim = 13

# Determine nhead from checkpoint
nhead_key = f'{PREFIX}transformer_decoder.layers.0.self_attn.in_proj_weight'
if nhead_key in dec_state:
    nhead = max(1, D_MODEL // 64)  # 1024/64=16 heads, 512/64=8 heads
else:
    nhead = 8 if D_MODEL == 512 else 16

# Determine max_len from checkpoint pos_encoding
pe_key = f'{PREFIX}pos_encoding.pe'
if pe_key in dec_state:
    ckpt_max_len = dec_state[pe_key].shape[1]
else:
    ckpt_max_len = 80

print(f"Detected from checkpoint:")
print(f"  vocab_size={ckpt_vocab_size}, stoich_input_dim={ckpt_stoich_dim}")
print(f"  d_model={D_MODEL}, dim_feedforward={DIM_FEEDFORWARD}, nhead={nhead}")
print(f"  pos_encoding max_len={ckpt_max_len}")

# Instantiate V15.0 decoder with auto-detected dimensions
# Use checkpoint's max_len to avoid pos_encoding.pe shape mismatch
decoder = EnhancedTransformerDecoder(
    latent_dim=2048,
    d_model=D_MODEL,
    nhead=nhead,
    num_layers=12,
    dim_feedforward=DIM_FEEDFORWARD,
    max_len=ckpt_max_len,
    n_memory_tokens=N_MEMORY_TOKENS,
    use_skip_connection=False,
    use_stoich_conditioning=True,
    n_stoich_tokens=4,
    vocab_size=ckpt_vocab_size,
    stoich_input_dim=ckpt_stoich_dim,
    memory_bottleneck_dim=BOTTLENECK_DIM,
)

# Strip compiled prefix if needed
load_state = dec_state
if PREFIX:
    load_state = {k.replace(PREFIX, ''): v for k, v in dec_state.items()}

# Filter out any remaining size-mismatched keys to avoid RuntimeError
# (strict=False only handles missing/unexpected, not shape mismatches)
model_state = decoder.state_dict()
filtered_state = {}
skipped = []
for k, v in load_state.items():
    if k in model_state and model_state[k].shape != v.shape:
        skipped.append((k, list(v.shape), list(model_state[k].shape)))
    else:
        filtered_state[k] = v

if skipped:
    print(f"\nSkipped {len(skipped)} size-mismatched keys:")
    for k, ckpt_shape, model_shape in skipped:
        print(f"  {k}: checkpoint {ckpt_shape} vs model {model_shape}")

missing, unexpected = decoder.load_state_dict(filtered_state, strict=False)

print(f"\nLoad results:")
if missing:
    print(f"  Missing keys ({len(missing)}):")
    for k in missing[:10]:
        print(f"    {k}")
    if len(missing) > 10:
        print(f"    ... and {len(missing)-10} more")
else:
    print(f"  Missing keys: None")
if unexpected:
    print(f"  Unexpected keys ({len(unexpected)}):")
    for k in unexpected[:10]:
        print(f"    {k}")
    if len(unexpected) > 10:
        print(f"    ... and {len(unexpected)-10} more")
else:
    print(f"  Unexpected keys: None")

# Param counts
ltm_params = sum(p.numel() for n, p in decoder.named_parameters() if 'latent_to_memory' in n)
total_params = sum(p.numel() for p in decoder.parameters())
print(f"\nParam counts:")
print(f"  latent_to_memory: {ltm_params:,}")
print(f"  Total decoder:    {total_params:,}")

# Quick forward pass test
z = torch.randn(2, 2048)
stoich = torch.randn(2, ckpt_stoich_dim)
heads_pred = {
    'tc_pred': torch.randn(2),
    'sc_pred': torch.randn(2),
    'hp_pred': torch.randn(2),
    'tc_class_logits': torch.randn(2, 5),
    'competence': torch.randn(2),
    'element_count_pred': torch.randn(2),
}
memory = decoder._create_memory(z, stoich_pred=stoich, heads_pred=heads_pred)
expected_total = N_MEMORY_TOKENS + 4 + 4  # latent + stoich + heads
print(f"\nMemory shape: {list(memory.shape)}")
print(f"  Expected: [2, {expected_total}, {D_MODEL}] = {N_MEMORY_TOKENS} latent + 4 stoich + 4 heads")
assert memory.shape == (2, expected_total, D_MODEL), f"Shape mismatch: {memory.shape} != (2, {expected_total}, {D_MODEL})"
print("\nVerification PASSED")

## Cell 10: Point checkpoint_best.pt to Migrated File

The training script's auto-resume looks for `checkpoint_best.pt`. This cell replaces it with the migrated version (the original is safe in the backup).

**Run this only after you've verified Cell 9 passes.**

In [ ]:
import shutil

best_path = CHECKPOINT_PATH.parent / "checkpoint_best.pt"

print(f"Replacing checkpoint_best.pt with migrated version...")
print(f"  Backup at: {BACKUP_PATH.name}")

# Copy migrated -> checkpoint_best.pt
shutil.copy2(str(OUTPUT_PATH), str(best_path))
print(f"  Done! checkpoint_best.pt is now the V15.0 migrated checkpoint.")
print(f"\nTo rollback: copy {BACKUP_PATH.name} back to checkpoint_best.pt")
print(f"\nReady to resume training! Run the train_colab.ipynb notebook.")

## Cell 11 (Optional): Quick 1-Epoch Smoke Test

Run 1 epoch of training to verify the full pipeline works end-to-end before committing to a long training run.

In [ ]:
import importlib

scripts_path = str(REPO_PATH / "scripts")
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

import train_v12_clean
importlib.reload(train_v12_clean)

# Override for 1-epoch smoke test
train_v12_clean.TRAIN_CONFIG['num_epochs'] = checkpoint.get('epoch', 0) + 1  # Just 1 more epoch
train_v12_clean.TRAIN_CONFIG['resume_checkpoint'] = 'auto'
train_v12_clean.TRAIN_CONFIG['checkpoint_interval'] = 1
train_v12_clean.TRAIN_CONFIG['use_gradient_checkpointing'] = False
train_v12_clean.TRAIN_CONFIG['z_cache_every_epoch'] = False

# Point to Drive paths
train_v12_clean.PROJECT_ROOT = REPO_PATH
train_v12_clean.CONTRASTIVE_DATA_PATH = REPO_PATH / 'data/processed/supercon_fractions_contrastive.csv'
train_v12_clean.DATA_PATH = REPO_PATH / 'data/processed/supercon_fractions_contrastive.csv'
train_v12_clean.HOLDOUT_PATH = REPO_PATH / 'data/GENERATIVE_HOLDOUT_DO_NOT_TRAIN.json'
train_v12_clean.OUTPUT_DIR = REPO_PATH / 'outputs'

print(f"Smoke test: running 1 epoch from epoch {checkpoint.get('epoch', 0)}")
print(f"If this completes without errors, the migration is confirmed working.\n")

train_v12_clean.train()